In [1]:
import os
import numpy as np
import glob
from tess_sip import SIP
from astropy.coordinates import SkyCoord
from astropy import units as u
import lightkurve as lk
from astroquery.mast import Tesscut
import tess_cpm
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

sys.path.insert(0, "../")
sys.path.insert(0, "../../Packages/eleanor/")
import eleanor

import warnings
warnings.filterwarnings('ignore')

from michael import janet

In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

index = uni[uni.source_id == int(2174289161681454720)].index[0]

In [3]:
j = janet.boot(uni, index=index)


In [4]:
coords = SkyCoord(ra = j.ra, dec = j.dec, unit = (u.deg, u.deg))


In [6]:
from michael.data import data_class
d = data_class(j)
d.setup_data()

Target DR3 ID 2174289161681454720 has tesscut data available on MAST for Sectors 16, 17, 56, 57
Data already downloaded for Sector 16.
Data already downloaded for Sector 17.
Data already downloaded for Sector 56.
Data already downloaded for Sector 57.


In [7]:
d.build_eleanor_lc()

In [ ]:
d.build_tess_sip_lc(detrended=True)

Running pixels in aperture: 100%|████████████| 300/300 [00:00<00:00, 398.05it/s]


In [ ]:
d.build_unpopular_lc()


In [ ]:
sfiles = []
for sector in j.sectors:
    split = sector.split('-')
    if len(split) > 1:
        sfiles = []
        for s in np.arange(int(split[0]), int(split[1])+1):
            strlen = np.floor(np.log10(s)).astype(int)+1
            secstr = 's0000'[:-strlen] + str(s)

            sfile = glob.glob(f'{d.path}'+
                                f'*{secstr}*astrocut.fits')

            if len(sfile) == 0:
                raise ValueError("No tesscut files could be found for this target.")

            sfiles.append(sfile[0])

        tpflist = [lk.TessTargetPixelFile(f).cutout([26,26],13) for f in sfiles]
        tpfs = lk.TargetPixelFileCollection(tpflist)

In [ ]:
img = tpfs[1].plot();
plt.show()

In [ ]:
lims = img.get_images()[0].get_clim()

In [ ]:
plt.imshow(j.void['datum_57'].tpf[0], origin='lower',
          vmin = lims[0], vmax = lims[1])
pix = np.where(j.void[f'datum_57'].aperture > 0)
plt.scatter(pix[0], pix[1], edgecolors='w', lw=5, marker=',', facecolors='none', s=600, zorder=2, label='Aperture')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i, m in enumerate(['c','r','cpm','rdt']):
    (j.void[f'{m}lc_16-17'] + i*.02).plot(ax=ax, label=m)
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i, m in enumerate(['c','r','cpm','rdt']):
    (j.void[f'{m}lc_56-57'] + i*.02).plot(ax=ax, label=m)
plt.show()

In [ ]:
fig,ax = plt.subplots()
for s in j.sectorlist:
    j.void[f'cpmlc_{s}'].plot()
    plt.show()    

In [ ]:
for s in j.sectors:
    if len(s.split('-')) > 1:
        sectors = np.arange(int(s.split('-')[0]), int(s.split('-')[-1])+1)

        cpmlc = j.void[f'cpmlc_{sectors[0]}']

        for i in sectors[1:]:
            cpmlc = cpmlc.append(j.void[f'cpmlc_{i}'])
        cpmlc.plot()
        plt.show()

In [ ]:
j.sfiles

In [ ]:
j.sectorlist

In [ ]:
glob.glob(f'{os.path.expanduser("~")}/.michael/tesscut/{j.gaiaid}/*astrocut.fits')

In [ ]:
sort = np.argsort([f.split('/')[-1].split('-')[1][1:] for f in j.sfiles])

In [ ]:
np.array(j.sfiles)[sort].tolist()

In [ ]:
sort